In [1]:
import seaborn as sns
import pandas as pd
import plotly.express as px
import numpy as np

### Extraemos la informacion a dataframes para su uso

In [ ]:
df = pd.read_csv("""DIRECTORIO""")
df_sp500 = pd.read_csv("""DIRECTORIO""")
df_bitcoin = pd.read_csv("""DIRECTORIO""")
df_dolar = pd.read_csv("""DIRECTORIO""")

#### Limpieza de los dataframes

In [4]:
df_dolar = df_dolar[['date','euro_to_usd']]
df_dolar.rename(columns={'date': 'Fecha', 'euro_to_usd': 'Euro'}, inplace=True)

In [5]:
df_bitcoin = df_bitcoin[['Open time','Open']]
df_bitcoin.rename(columns={'Open time': 'Fecha', 'Open': 'Precio_Bitcoin'}, inplace=True)

In [6]:
df_sp500 = df_sp500[['Date','Open']]
df_sp500.rename(columns={'Date': 'Fecha','Open': 'Precio_S&P500'}, inplace=True)

#### Filtramos solo las categorias de "Ahorro" que seran las que utilizaremos

In [7]:
df = df[df["Categoria"]=='ahorro']

#### Nos aseguremos que todos los campos de fecha esten en el formato correcto

In [8]:
#Convierto la columna fecha a formato fecha
df['Fecha'] = df['Fecha'].astype('datetime64[ns]')
df_sp500['Fecha'] = df_sp500['Fecha'].astype('datetime64[ns]')
df_bitcoin['Fecha'] = df_bitcoin['Fecha'].astype('datetime64[ns]')
df_dolar['Fecha'] = df_dolar['Fecha'].astype('datetime64[ns]')


#### Comrpobamos las limpiezas 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 4500 to 4562
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha      63 non-null     datetime64[ns]
 1   Importe    63 non-null     float64       
 2   Categoria  63 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 2.0+ KB


In [10]:
df_bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2626 entries, 0 to 2625
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Fecha           2626 non-null   datetime64[ns]
 1   Precio_Bitcoin  2626 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 41.2 KB


In [11]:
df_sp500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24410 entries, 0 to 24409
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Fecha          24410 non-null  datetime64[ns]
 1   Precio_S&P500  24410 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 381.5 KB


In [12]:
df_dolar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5323 entries, 0 to 5322
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Fecha   5323 non-null   datetime64[ns]
 1   Euro    5255 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 83.3 KB


### Merge de todos los datos a un dataframe

In [13]:
Precios_df = df.merge(df_sp500, left_on='Fecha', right_on='Fecha', how='left')
Precios_df = Precios_df.merge(df_bitcoin, left_on='Fecha', right_on='Fecha', how='left')
Precios_df = Precios_df.merge(df_dolar, left_on='Fecha', right_on='Fecha', how='left')

#### Llenamos los datos vacios con los superiores e inferiores 

In [ ]:
# Ordenar el dataframe por la columna 'Fecha'
Precios_df = Precios_df.sort_values(by='Fecha')

#Al ser historicos de precios, hay campos vacios, en especial festivos y fines de semana
Precios_df['Euro'] = Precios_df['Euro'].ffill()
Precios_df['Precio_Bitcoin'] = Precios_df['Precio_Bitcoin'].ffill()
Precios_df['Precio_S&P500'] = Precios_df['Precio_S&P500'].ffill()
Precios_df['Euro'] = Precios_df['Euro'].bfill()
Precios_df['Precio_Bitcoin'] = Precios_df['Precio_Bitcoin'].bfill()
Precios_df['Precio_S&P500'] = Precios_df['Precio_S&P500'].bfill()


#### Convertimos nuestros precios de bitcoin y S&P500 a euros con el cambio correspondient al dia

In [15]:
Precios_df['Precio_Bitcoin'] = Precios_df['Precio_Bitcoin']*Precios_df['Euro']
Precios_df['Precio_S&P500'] = Precios_df['Precio_S&P500']*Precios_df['Euro']

In [16]:
Precios_df = Precios_df.drop('Euro', axis=1)

In [17]:
Precios_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, 32 to 0
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Fecha           63 non-null     datetime64[ns]
 1   Importe         63 non-null     float64       
 2   Categoria       63 non-null     object        
 3   Precio_S&P500   63 non-null     float64       
 4   Precio_Bitcoin  63 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 3.0+ KB


In [ ]:
Precios_df.to_csv("""DIRECTORIO""")

### Funcion con la que creamos la simulacion, por entrada

In [21]:
def simulate_investment_evolution(Bitcoin, Fondo, df = Precios_df):
    Efectivo = (100-(Fondo + Bitcoin))/100
    Bitcoin = Bitcoin / 100
    Fondo = Fondo / 100
    df = df.copy()
    df['Importe'] = df['Importe'].abs()
    df['Efectivo'] = df['Importe'] * Efectivo
    df['sp_units'] = (df['Importe'] * Fondo) / df['Precio_S&P500']
    df['btc_units'] = (df['Importe'] * Bitcoin) / df['Precio_Bitcoin']
    
    df['Acum_Efectivo'] = df['Efectivo'].cumsum()
    df['Acum_sp_units'] = df['sp_units'].cumsum()
    df['Acum_btc_units'] = df['btc_units'].cumsum()

    df['Total_ganado'] = df['Acum_Efectivo'] + df['Acum_sp_units'] * df['Precio_S&P500'] + df['Acum_btc_units'] * df['Precio_Bitcoin']
    
    return df[['Fecha', 'Total_ganado']]

### Combinamos las distintas distribuciones con la simulacion de precios


In [22]:
def simulate_investment_evolution_for_all_options(OpcionesDF, Datos_DF):
    results = {}
    
    for index, row in OpcionesDF.iterrows():
        btc = row['BTC']
        sp = row['SP']
        investment_evolution = simulate_investment_evolution(btc, sp, Datos_DF)
        results[(btc, sp)] = investment_evolution
    
    results = pd.concat(results, names=["BTC", "SP"]).reset_index(level=[0, 1])
    
    return results

### Creamos un dataframe con todas las opciones de inversion

In [ ]:
# Crear los valores para BTC y SP, en este caso, de 10 en 10, pero podemos variar el rango
btc_values = list(range(0, 101,10)) 
sp_values = list(range(0, 101,10))

# Crear una lista para almacenar las combinaciones
data = []

# Utilizar bucles for para generar las combinaciones
for btc in btc_values:
    for sp in sp_values:
        if btc + sp <= 100:
            data.append((btc, sp))

# Crear el DataFrame a partir de la lista de combinaciones
DF_Opciones_Inversion = pd.DataFrame(data, columns=['BTC', 'SP'])

### Ejecutamos la simulacion

In [32]:
Resultados_DF = simulate_investment_evolution_for_all_options(DF_Opciones_Inversion, Precios_df)
Resultados_DF



,BTC,SP,Fecha,Total_ganado
32,0,0,2019-06-01,250.000000
31,0,0,2019-07-01,500.000000
30,0,0,2019-07-29,510.000000
29,0,0,2019-07-31,530.000000
28,0,0,2019-08-01,780.000000
...,...,...,...,...
52,100,0,2023-08-01,42090.633276
62,100,0,2023-12-05,60768.428058
16,100,0,2024-03-01,89034.465059
15,100,0,2024-03-07,97463.574717


### Obtenemos todas las opciones para la ultima fecha del dataframe

In [33]:
# Obtener la última fecha
ultima_fecha = Resultados_DF['Fecha'].max()

# Filtrar el DataFrame para obtener solo las filas correspondientes a la última fecha
df_ultima_fecha = Resultados_DF[Resultados_DF['Fecha'] == ultima_fecha]

### Grafico temporal de algunas de las opciones de inversion

In [43]:
# Asegurarse de que la columna 'Fecha' esté en formato datetime
Resultados_DF['Fecha'] = pd.to_datetime(Resultados_DF['Fecha'])

# Crear una nueva columna para identificar cada distribución de inversión
Resultados_DF['Distribucion'] = Resultados_DF.apply(
    lambda row: f"BTC {row['BTC']}% - SP {row['SP']}%", axis=1
)

Resultados_DF = Resultados_DF[Resultados_DF['Distribucion'].isin(['BTC 0% - SP 0%', 'BTC 50% - SP 50%','BTC 100% - SP 0%','BTC 0% - SP 100%'])] #Filtro distribuciones interesantes


# Crear el gráfico de líneas con markers para cada distribución
fig = px.line(
    Resultados_DF, 
    x='Fecha', 
    y='Total_ganado', 
    color='Distribucion', 
    markers=True,
    title='Evolución Temporal de la Inversión',
    labels={'Total_ganado': 'Total Ganado', 'Fecha': 'Fecha'},
    template="ggplot2"
)

# Ajustar la leyenda para que aparezca fuera del gráfico (a la derecha)
fig.update_layout(
    legend=dict(
        title='Distribución de Inversión',
        x=1.05,  # Posición horizontal fuera del gráfico
        y=1,     # Posición vertical (parte superior)
        xanchor='left'
    )
)


# Mostrar el gráfico interactivo
fig.show()

### Creamos la matriz, comparando las opciones de inversion a la ultima fecha del dataframe

In [44]:
matriz = pd.crosstab(index=df_ultima_fecha['SP'], columns=df_ultima_fecha['BTC'], values=df_ultima_fecha['Total_ganado'], aggfunc='sum')

In [45]:
matriz

BTC,0,10,20,30,40,50,60,70,80,90,100
SP,,,,,,,,,,,
0,29670.190000,35971.149601,42272.109201,48573.068802,54874.028403,61174.988003,67475.947604,73776.907205,80077.866805,86378.826406,92679.786007
10,30406.496597,36707.456197,43008.415798,49309.375399,55610.334999,61911.294600,68212.254201,74513.213801,80814.173402,87115.133003,NaN
20,31142.803193,37443.762794,43744.722395,50045.681995,56346.641596,62647.601197,68948.560797,75249.520398,81550.479999,NaN,NaN
30,31879.109790,38180.069391,44481.028991,50781.988592,57082.948193,63383.907793,69684.867394,75985.826995,NaN,NaN,NaN
40,32615.416387,38916.375987,45217.335588,51518.295189,57819.254789,64120.214390,70421.173991,NaN,NaN,NaN,NaN
50,33351.722983,39652.682584,45953.642185,52254.601785,58555.561386,64856.520987,NaN,NaN,NaN,NaN,NaN
60,34088.029580,40388.989181,46689.948781,52990.908382,59291.867983,NaN,NaN,NaN,NaN,NaN,NaN
70,34824.336177,41125.295777,47426.255378,53727.214979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,35560.642773,41861.602374,48162.561975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
styled_matriz = (
    matriz.style
    .background_gradient(cmap='viridis', axis=None)  # Se asegura de aplicar el gradiente a todo el DataFrame
    .format(na_rep='')  # Oculta los valores NaN
)

# Mostrar matriz estilizada
styled_matriz


BTC,0,10,20,30,40,50,60,70,80,90,100
SP,,,,,,,,,,,
0,29670.190000,35971.149601,42272.109201,48573.068802,54874.028403,61174.988003,67475.947604,73776.907205,80077.866805,86378.826406,92679.786007
10,30406.496597,36707.456197,43008.415798,49309.375399,55610.334999,61911.294600,68212.254201,74513.213801,80814.173402,87115.133003,
20,31142.803193,37443.762794,43744.722395,50045.681995,56346.641596,62647.601197,68948.560797,75249.520398,81550.479999,,
30,31879.109790,38180.069391,44481.028991,50781.988592,57082.948193,63383.907793,69684.867394,75985.826995,,,
40,32615.416387,38916.375987,45217.335588,51518.295189,57819.254789,64120.214390,70421.173991,,,,
50,33351.722983,39652.682584,45953.642185,52254.601785,58555.561386,64856.520987,,,,,
60,34088.029580,40388.989181,46689.948781,52990.908382,59291.867983,,,,,,
70,34824.336177,41125.295777,47426.255378,53727.214979,,,,,,,
80,35560.642773,41861.602374,48162.561975,,,,,,,,


### HeatMap con la matriz para visualizar la evolucion de la inversion segun las opciones

In [59]:
fig = px.imshow(
    matriz, 
    #text_auto=True, 
    text_auto=".2f",
    color_continuous_scale="Viridis" 
)

fig.update_layout(
    title="Distribucion de Inversion S&P500 vs Bitcoin",  
    xaxis_title="Bitcoin",  
    yaxis_title="S&P500",  
    template="ggplot2", 
    width=800,  
    height=600,  
    margin=dict(
        l=10,  # izquierdo
        r=10,  # derecho
        t=50,  # superior
        b=10   #  inferior
    )
)

fig.show()
